# Draftkings Over/Unders

In [ ]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "Priority": "u=0, i",
    "Sec-CH-UA": "\"Not)A;Brand\";v=\"99\", \"Google Chrome\";v=\"127\", \"Chromium\";v=\"127\"",
    "Sec-CH-UA-Mobile": "?0",
    "Sec-CH-UA-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

subcategories = {
    'passing_yards':'7200',
    'passing_td':'14770',
    'rec_yards':'7276',
    'rec_td':'7239',
    'rush_yards':'7277',
    'rush_td':'7694',
    'rec':'13405',
    'int':'13350',
    'OROY':'13342',
    'OPOY':'13340',
}

In [ ]:
def clean_name(name):
    return re.sub(r'[\.]|([SJ]r)|I+$','',name).strip()

def get_market_id(markets,event_id):
    for market in markets:
        if market.get('eventId') == event_id:
            return market['id']
        
def get_label(selections,market_id):
    for selection in selections:
        if selection.get('marketId') == market_id:
            return selection['label']
        
def add_field(odds_dict,name,key,val):
    existing_odds = odds_dict.get(name,{})
    existing_odds[key] = val
    odds_dict[name] = existing_odds
    return odds_dict

def create_odds_dict_single_market(response):
    page_dict = response.json()
    selections = page_dict['selections']
    odds_dict = {}
    for s in selections:
        name = clean_name(s['label'])
        over_under = s['displayOdds']['american']
        odds_dict[name]=over_under
    return odds_dict

def create_odds_dict(response):
    page_dict = response.json()
    markets = page_dict['markets']
    selections = page_dict['selections']
    odds_dict = {}
    events = page_dict['events']
    for event in events:
        #identify names
        if any(id_type in event for id_type in ['eventId','marketId']):
            continue

        event_name = event['name']
        participants = event['participants']
        for p in participants:
            if (participant_name:=clean_name(p['name'])) in clean_name(event_name):
                break
        if not participant_name:
            print(event)
        event_id = event['id']
        market_id = get_market_id(markets,event_id)
        label = get_label(selections,market_id)
        over_under = float(label.split(' ',1)[1])
        odds_dict[participant_name] = over_under
    return odds_dict

In [ ]:
agg_odds_dict = {}
for subcategory_name,subcategory_id in subcategories.items():
    if subcategory_name in ['OROY','OPOY']:
        single_market = True
    else:
        single_market = False
    url = f'https://sportsbook-nash.draftkings.com/api/sportscontent/dkusco/v1/leagues/88808/categories/782/subcategories/{subcategory_id}'
    
    if single_market:
        url = url.replace('782','787')
        create_odds_dict_func = create_odds_dict_single_market
    else:
        create_odds_dict_func = create_odds_dict
        
    response = requests.get(url,headers=headers)
    odds_dict = create_odds_dict_func(response)
    
    for name,over_under in odds_dict.items():
        agg_odds_dict = add_field(agg_odds_dict,name,key=subcategory_name,val=over_under)

In [ ]:
df_dk = pd.DataFrame(data=agg_odds_dict).transpose().reset_index()
df_dk.rename(columns={'index': 'Player'}, inplace=True)
df_dk.to_clipboard(sep='\t',index=False)
df_dk.head()

# ESPN Projections

In [ ]:
ESPN_MAPPING = {
    '3':'passing_yards',
    '4':'passing_td',
    '20':'int',
    '24':'rush_yards',
    '25':'rush_td',
    '42':'rec_yards',
    '43':'rec_td',
    '53':'rec',
}
POSITION_MAPPING = {
    1:'QB',
    2:'RB',
    3:'WR',
    4:'TE',
    5:'K',
    16:'D/ST',
}

In [ ]:
url = 'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/2024/segments/0/leaguedefaults/3?scoringPeriodId=0&view=kona_player_info'
headers = {
    'X-Fantasy-Filter': '{"players":{"filterSlotIds":{"value":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,23,24]},"filterStatsForExternalIds":{"value":[2023,2024]},"filterStatsForSourceIds":{"value":[0,1]},"sortAppliedStatTotal":{"sortAsc":false,"sortPriority":3,"value":"102024"},"sortDraftRanks":{"sortPriority":2,"sortAsc":true,"value":"PPR"},"sortPercOwned":{"sortAsc":false,"sortPriority":4},"limit":5000,"offset":0,"filterRanksForScoringPeriodIds":{"value":[1]},"filterRanksForRankTypes":{"value":["PPR"]},"filterRanksForSlotIds":{"value":[0,2,4,6,17,16,8,9,10,12,13,24,11,14,15]},"filterStatsForTopScoringPeriodIds":{"value":2,"additionalValue":["002024","102024","002023","022024"]}}}',
}

In [ ]:
response = requests.get(url,headers=headers)
players = response.json()['players']

odds_dict = {}
for player in players:
    position_id = player['player']['defaultPositionId']
    position_name = POSITION_MAPPING[position_id]
    if position_id in [5,16]:
        continue

    name = player['player']['fullName']
    name = clean_name(name)
    
    stats_obj = player['player']['stats']
    current_year = str(datetime.now().year)
    filtered_stats_obj = [s for s in stats_obj if s['externalId'] == current_year and s.get('stats')]
    if not filtered_stats_obj:
        continue
        
    stats = filtered_stats_obj[0]['stats']
    for col_id,col_name in ESPN_MAPPING.items():
        if (val:=stats.get(col_id)):
            odds_dict = add_field(odds_dict,name,key=col_name,val=val)
            
    ppr_vals = player['player']['draftRanksByRankType']['PPR']
    auction_val = ppr_vals['auctionValue']
    rank = ppr_vals['rank']
    
    odds_dict = add_field(odds_dict,name,key='auction_val',val=auction_val)
    odds_dict = add_field(odds_dict,name,key='ppr_rank',val=rank)
    odds_dict = add_field(odds_dict,name,key='position',val=position_name)

In [ ]:
df_espn = pd.DataFrame(odds_dict).transpose().reset_index()
df_espn.rename(columns={'index': 'Player'}, inplace=True)
df_espn.sort_values('auction_val',ignore_index=True,ascending=False,inplace=True)
df_espn.to_clipboard(sep='\t',index=False)

# Merge sources

In [ ]:
df_merged = pd.merge(how='outer', left=df_dk, right=df_espn, on='Player',suffixes=('_DK','_ESPN'))
df_merged.fillna(0,inplace=True)

#add diff columns
for col in df_merged.columns:
    if not str(col).endswith('_DK'):
        continue
    base_col_name = str(col).replace('_DK','')
    diff_col_name = base_col_name + '_zdiff'
    df_merged[diff_col_name] = 0

sorted_cols = list(df_merged.columns)
sorted_cols.sort()
end_columns = ['ppr_rank','auction_val','OPOY','OROY']
start_columns = ['Player','position']
sorted_cols = start_columns + [col for col in sorted_cols if col not in end_columns + start_columns] + end_columns
df_merged = df_merged[sorted_cols].copy()

rename_dict = {}
for col in df_merged.columns:
    if '_zdiff' in col:
        rename_dict[col] = col.replace('_zdiff','_diff')

df_merged.rename(columns=rename_dict,inplace=True)

In [ ]:
for diff_col in df_merged.columns:
    if not str(diff_col).endswith('_diff'):
        continue
    
    dk_col = diff_col.replace('_diff','_DK')
    espn_col = diff_col.replace('_diff','_ESPN')
    df_merged[diff_col] = df_merged.apply(lambda x:0 if not x[dk_col] else x[dk_col] - x[espn_col],axis=1)
    
df_merged.to_clipboard(sep='\t')

In [ ]:
SCORING = {
    'passing_td' : 4,
    'int' : -2,
    'rec_td' : 6,
    'rec_yards' : 0.1,
    'rush_yards' : 0.1,
    'rush_td' : 6,
    'rec' : 0.5,
    'passing_yards' : 0.04,
}

df_merged['comment'] = ''
for i,row in df_merged.iterrows():
    diff_list = []
    diff_pts = 0
    
    for score_cat,score_pts in SCORING.items():
        diff_col = score_cat + '_diff'
        if not (diff_amt:=row[diff_col]):
            continue
        diff_list.append((score_cat,diff_amt*score_pts,diff_amt))
    
    if not diff_list:
        continue
        
    diff_pts = sum([t[1] for t in diff_list])
    max_tuple = max(diff_list,key=lambda x:abs(x[1]))

    if abs(diff_pts) < 10:
        continue
        
    if max_tuple[2] > 0:
        est = 'overestimated'
    else:
        est = 'underestimated'
        
    comment = f'{max_tuple[0]} {est} by {abs(int(max_tuple[2]))}'
    df_merged.loc[i,'comment'] = comment
    

df_merged.to_clipboard(sep='\t',index=False)        